In [12]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [13]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [14]:
subject = 'Distrito Federal - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [15]:
data = pd.read_csv('2003_mo_model_input_DF.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Distrito Federal - value,Distrito Federal - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - IDH,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - Produção de Cimento (t),Distrito Federal - Consumo de Cimento (t)
0,2003-1,0.254227,8.293882,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.826971,1.184017e+08,3.570616e+06,43.639430,1.092465e+08,156.625720,50.047
1,2003-2,0.252655,8.287887,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.827063,1.185511e+08,3.573184e+06,43.651300,1.093025e+08,157.678528,48.110
2,2003-3,0.250972,8.281892,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.827155,1.187006e+08,3.575752e+06,43.663170,1.093585e+08,159.190268,49.006
3,2003-4,0.249179,8.275896,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.827247,1.188500e+08,3.578320e+06,43.675041,1.094145e+08,160.688376,48.445
4,2003-5,0.247070,8.269901,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.827340,1.189994e+08,3.580889e+06,43.686911,1.094705e+08,162.847410,51.436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,0.529279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250.077918,77.448
236,2022-9,0.527896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,248.545664,64.321
237,2022-10,0.526069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,248.265413,72.839
238,2022-11,0.523943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247.486640,59.598


In [16]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Distrito Federal - value,Distrito Federal - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - IDH,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - Produção de Cimento (t)
0,-0.765895,-0.830412,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.593530,-1.693822,0.396337,-2.171734,-2.205731,-1.931775
1,-0.800114,-0.833674,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.541888,-1.674984,0.431282,-2.081307,-2.163694,-1.908347
2,-0.836745,-0.836936,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.490246,-1.656145,0.466227,-1.990880,-2.121657,-1.874707
3,-0.875764,-0.840198,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.438604,-1.637307,0.501171,-1.900453,-2.079620,-1.841370
4,-0.921678,-0.843461,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.386962,-1.618468,0.536116,-1.810026,-2.037583,-1.793327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.298634,1.198041,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.236110,1.116166,-1.071950,-1.286229,0.817896,-0.512069
188,1.354774,1.198255,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.183845,1.104850,-1.047047,-1.328476,0.799100,-0.500824
189,1.401307,1.198470,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.131580,1.093534,-1.022144,-1.370723,0.780305,-0.491677
190,1.460103,1.198684,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.079315,1.082218,-0.997242,-1.412970,0.761510,-0.481495


In [17]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      46.442
1      43.234
2      54.587
3      59.228
4      63.997
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Distrito Federal - Consumo de Cimento (t), Length: 240, dtype: float64

In [18]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Distrito Federal - value,Distrito Federal - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - IDH,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - Produção de Cimento (t)
0,-0.765895,-0.830412,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.593530,-1.693822,0.396337,-2.171734,-2.205731,-1.931775
1,-0.800114,-0.833674,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.541888,-1.674984,0.431282,-2.081307,-2.163694,-1.908347
2,-0.836745,-0.836936,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.490246,-1.656145,0.466227,-1.990880,-2.121657,-1.874707
3,-0.875764,-0.840198,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.438604,-1.637307,0.501171,-1.900453,-2.079620,-1.841370
4,-0.921678,-0.843461,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.386962,-1.618468,0.536116,-1.810026,-2.037583,-1.793327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.937260,1.309227,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.616226,1.235244,-1.506889,-0.747477,1.174551,0.131039
158,0.925760,1.302282,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.563561,1.236423,-1.511843,-0.763990,1.168128,0.077457
159,0.912813,1.295337,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.510895,1.237602,-1.516798,-0.780503,1.161706,0.032090
160,0.906620,1.288392,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.458230,1.238780,-1.521752,-0.797016,1.155283,-0.032489


In [19]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      46.442
1      43.234
2      54.587
3      59.228
4      63.997
        ...  
157    35.599
158    49.981
159    39.881
160    48.598
161    49.942
Name: Distrito Federal - Consumo de Cimento (t), Length: 162, dtype: float64

In [20]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Distrito Federal - value,Distrito Federal - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - IDH,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - Produção de Cimento (t)
162,0.902328,1.274502,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,1.352899,1.241137,-1.531661,-0.830042,1.142438,-0.088054
163,0.902814,1.267557,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,1.300233,1.242316,-1.536615,-0.846555,1.136015,-0.105352
164,0.907317,1.260612,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,1.247568,1.243494,-1.541569,-0.863068,1.129592,-0.117887
165,0.914492,1.253668,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,1.194903,1.244673,-1.546523,-0.879581,1.123170,-0.134394
166,0.915160,1.246723,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,1.142237,1.245851,-1.551478,-0.896095,1.116747,-0.163790
167,0.914118,1.239778,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,1.089572,1.247030,-1.556432,-0.912608,1.110324,-0.197707
168,0.913548,1.232833,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,1.036906,1.248208,-1.561386,-0.929121,1.103902,-0.234373
169,0.922162,1.229808,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,1.000661,1.243806,-1.535126,-0.934236,1.091032,-0.271993
170,0.931366,1.226784,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,0.964416,1.239403,-1.508866,-0.939351,1.078162,-0.289415
171,0.942855,1.223760,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,0.928171,1.235001,-1.482607,-0.944465,1.065292,-0.315182


In [21]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    53.235
163    59.290
164    54.875
165    57.520
166    49.573
167    40.698
168    47.195
169    43.527
170    45.488
171    44.960
172    36.721
173    54.463
174    51.462
175    58.854
176    53.940
177    51.514
178    42.635
179    40.371
180    48.990
181    44.726
182    44.491
183    43.901
184    55.019
185    50.942
186    63.992
187    64.127
188    60.822
189    65.182
190    51.282
191    43.554
Name: Distrito Federal - Consumo de Cimento (t), dtype: float64

In [22]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [23]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [26]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [27]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[879734783, 3465453232, 1629244912, 375501388, 1585315282, 3678478761, 3739960358, 3420587644, 1727912013, 2179788616, 502870164, 3584597009, 1742210170, 2008537515, 3690316247, 3505218498, 2486856343, 1149087968, 573820605, 3919907720, 3417328661, 2341142796, 1896535414, 3392300536, 108364276]


Step: 0 ___________________________________________
val_loss: 1916.2340087890625
winner_seed: 879734783


Step: 1 ___________________________________________
val_loss: 3336.59375


Step: 2 ___________________________________________
val_loss: 669.0719604492188
winner_seed: 1629244912


Step: 3 ___________________________________________
val_loss: 6100.04052734375


Step: 4 ___________________________________________
val_loss: 4138.48388671875


Step: 5 ___________________________________________
val_loss: 75.11467742919922
winner_seed: 3678478761


Step: 6 ___________________________________________
val_loss: 3457.427001953125


Step: 7 ___________________________________________
val_loss: 335

In [28]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 27ms/step - loss: 7659.2300 - val_loss: 2178.2686
Epoch 2/10000
5/5 [==============================] - 0s 6ms/step - loss: 6861.9121 - val_loss: 886.5637
Epoch 3/10000
5/5 [==============================] - 0s 7ms/step - loss: 5732.6172 - val_loss: 20.1030
Epoch 4/10000
5/5 [==============================] - 0s 6ms/step - loss: 5109.7192 - val_loss: 2308.0940
Epoch 5/10000
5/5 [==============================] - 0s 6ms/step - loss: 5069.8345 - val_loss: 1292.2512
Epoch 6/10000
5/5 [==============================] - 0s 6ms/step - loss: 4381.4321 - val_loss: 45.4358
Epoch 7/10000
5/5 [==============================] - 0s 7ms/step - loss: 4281.3330 - val_loss: 48.6539
Epoch 8/10000
5/5 [==============================] - 0s 7ms/step - loss: 3059.1467 - val_loss: 216.1776
Epoch 9/10000
5/5 [==============================] - 0s 7ms/step - loss: 2198.4846 - val_loss: 264.5623
Epoch 10/10000
5/5 [==============================] - 0s 6ms/st

5/5 [==============================] - 0s 7ms/step - loss: 142.2085 - val_loss: 1469.6660
Epoch 80/10000
5/5 [==============================] - 0s 6ms/step - loss: 145.1811 - val_loss: 1200.7773
Epoch 81/10000
5/5 [==============================] - 0s 6ms/step - loss: 160.4624 - val_loss: 1349.2264
Epoch 82/10000
5/5 [==============================] - 0s 6ms/step - loss: 155.1261 - val_loss: 1519.3068
Epoch 83/10000
5/5 [==============================] - 0s 6ms/step - loss: 161.3419 - val_loss: 1299.4008
Epoch 84/10000
5/5 [==============================] - 0s 5ms/step - loss: 156.6958 - val_loss: 1371.1373
Epoch 85/10000
5/5 [==============================] - 0s 5ms/step - loss: 152.6587 - val_loss: 1269.8807
Epoch 86/10000
5/5 [==============================] - 0s 5ms/step - loss: 136.1352 - val_loss: 1181.9546
Epoch 87/10000
5/5 [==============================] - 0s 6ms/step - loss: 139.3244 - val_loss: 1399.5055
Epoch 88/10000
5/5 [==============================] - 0s 6ms/step - lo

5/5 [==============================] - 0s 6ms/step - loss: 195.7474 - val_loss: 1328.2399
Epoch 157/10000
5/5 [==============================] - 0s 6ms/step - loss: 190.5031 - val_loss: 1569.2994
Epoch 158/10000
5/5 [==============================] - 0s 6ms/step - loss: 189.9672 - val_loss: 1342.7117
Epoch 159/10000
5/5 [==============================] - 0s 5ms/step - loss: 179.7842 - val_loss: 1432.1437
Epoch 160/10000
5/5 [==============================] - 0s 5ms/step - loss: 195.7169 - val_loss: 1363.6985
Epoch 161/10000
5/5 [==============================] - 0s 6ms/step - loss: 181.0934 - val_loss: 1287.0641
Epoch 162/10000
5/5 [==============================] - 0s 6ms/step - loss: 182.4359 - val_loss: 1360.0018
Epoch 163/10000
5/5 [==============================] - 0s 6ms/step - loss: 190.1799 - val_loss: 985.7468
Epoch 164/10000
5/5 [==============================] - 0s 5ms/step - loss: 196.8712 - val_loss: 1744.8494
Epoch 165/10000
5/5 [==============================] - 0s 6ms/s

5/5 [==============================] - 0s 5ms/step - loss: 204.5300 - val_loss: 1614.3384
Epoch 234/10000
5/5 [==============================] - 0s 6ms/step - loss: 186.3676 - val_loss: 1747.0363
Epoch 235/10000
5/5 [==============================] - 0s 5ms/step - loss: 184.2350 - val_loss: 1702.5094
Epoch 236/10000
5/5 [==============================] - 0s 5ms/step - loss: 173.2427 - val_loss: 1429.6003
Epoch 237/10000
5/5 [==============================] - 0s 5ms/step - loss: 173.7255 - val_loss: 1501.4092
Epoch 238/10000
5/5 [==============================] - 0s 5ms/step - loss: 178.7115 - val_loss: 1797.7407
Epoch 239/10000
5/5 [==============================] - 0s 6ms/step - loss: 180.9014 - val_loss: 1451.6255
Epoch 240/10000
5/5 [==============================] - 0s 5ms/step - loss: 177.7552 - val_loss: 1732.5878
Epoch 241/10000
5/5 [==============================] - 0s 6ms/step - loss: 176.3184 - val_loss: 1446.8206
Epoch 242/10000
5/5 [==============================] - 0s 5ms/

5/5 [==============================] - 0s 5ms/step - loss: 117.9344 - val_loss: 902.9748
Epoch 311/10000
5/5 [==============================] - 0s 5ms/step - loss: 121.4611 - val_loss: 682.7228
Epoch 312/10000
5/5 [==============================] - 0s 5ms/step - loss: 126.7157 - val_loss: 873.8668
Epoch 313/10000
5/5 [==============================] - 0s 6ms/step - loss: 132.1918 - val_loss: 1292.0049
Epoch 314/10000
5/5 [==============================] - 0s 6ms/step - loss: 111.4205 - val_loss: 1109.7976
Epoch 315/10000
5/5 [==============================] - 0s 5ms/step - loss: 115.3297 - val_loss: 828.1711
Epoch 316/10000
5/5 [==============================] - 0s 6ms/step - loss: 120.0857 - val_loss: 1046.5300
Epoch 317/10000
5/5 [==============================] - 0s 5ms/step - loss: 107.4129 - val_loss: 1082.7234
Epoch 318/10000
5/5 [==============================] - 0s 5ms/step - loss: 122.9880 - val_loss: 1525.8176
Epoch 319/10000
5/5 [==============================] - 0s 5ms/step

...

5/5 [==============================] - 0s 5ms/step - loss: 98.4018 - val_loss: 1054.4489
Epoch 1241/10000
5/5 [==============================] - 0s 5ms/step - loss: 98.5631 - val_loss: 1156.1173
Epoch 1242/10000
5/5 [==============================] - 0s 5ms/step - loss: 108.6203 - val_loss: 1235.5966
Epoch 1243/10000
5/5 [==============================] - 0s 5ms/step - loss: 106.8997 - val_loss: 1232.7080
Epoch 1244/10000
5/5 [==============================] - 0s 5ms/step - loss: 89.4823 - val_loss: 1009.6275
Epoch 1245/10000
5/5 [==============================] - 0s 5ms/step - loss: 89.4925 - val_loss: 1168.4265
Epoch 1246/10000
5/5 [==============================] - 0s 5ms/step - loss: 94.3600 - val_loss: 964.8827
Epoch 1247/10000
5/5 [==============================] - 0s 5ms/step - loss: 115.2776 - val_loss: 1184.9141
Epoch 1248/10000
5/5 [==============================] - 0s 5ms/step - loss: 105.7134 - val_loss: 1159.5833
Epoch 1249/10000
5/5 [==============================] - 0s 5

5/5 [==============================] - 0s 5ms/step - loss: 95.7877 - val_loss: 726.4137
Epoch 1318/10000
5/5 [==============================] - 0s 6ms/step - loss: 122.1149 - val_loss: 920.8687
Epoch 1319/10000
5/5 [==============================] - 0s 6ms/step - loss: 117.9065 - val_loss: 768.0355
Epoch 1320/10000
5/5 [==============================] - 0s 5ms/step - loss: 107.6276 - val_loss: 718.8338
Epoch 1321/10000
5/5 [==============================] - 0s 5ms/step - loss: 101.8962 - val_loss: 713.6875
Epoch 1322/10000
5/5 [==============================] - 0s 5ms/step - loss: 100.3961 - val_loss: 874.5467
Epoch 1323/10000
5/5 [==============================] - 0s 6ms/step - loss: 125.7652 - val_loss: 936.6619
Epoch 1324/10000
5/5 [==============================] - 0s 5ms/step - loss: 149.2570 - val_loss: 512.2734
Epoch 1325/10000
5/5 [==============================] - 0s 5ms/step - loss: 95.9167 - val_loss: 951.2847
Epoch 1326/10000
5/5 [==============================] - 0s 5ms/st

5/5 [==============================] - 0s 6ms/step - loss: 99.8595 - val_loss: 769.1856
Epoch 1395/10000
5/5 [==============================] - 0s 6ms/step - loss: 100.6349 - val_loss: 775.5244
Epoch 1396/10000
5/5 [==============================] - 0s 6ms/step - loss: 118.9334 - val_loss: 884.4700
Epoch 1397/10000
5/5 [==============================] - 0s 6ms/step - loss: 96.1584 - val_loss: 778.4645
Epoch 1398/10000
5/5 [==============================] - 0s 6ms/step - loss: 94.0768 - val_loss: 719.0807
Epoch 1399/10000
5/5 [==============================] - 0s 5ms/step - loss: 102.6988 - val_loss: 846.8459
Epoch 1400/10000
5/5 [==============================] - 0s 5ms/step - loss: 115.3686 - val_loss: 563.5644
Epoch 1401/10000
5/5 [==============================] - 0s 5ms/step - loss: 103.8798 - val_loss: 705.4351
Epoch 1402/10000
5/5 [==============================] - 0s 5ms/step - loss: 103.1491 - val_loss: 974.5199
Epoch 1403/10000
5/5 [==============================] - 0s 5ms/ste

Epoch 1472/10000
5/5 [==============================] - 0s 7ms/step - loss: 101.3384 - val_loss: 689.4987
Epoch 1473/10000
5/5 [==============================] - 0s 6ms/step - loss: 88.2829 - val_loss: 607.4863
Epoch 1474/10000
5/5 [==============================] - 0s 6ms/step - loss: 79.5547 - val_loss: 527.0098
Epoch 1475/10000
5/5 [==============================] - 0s 6ms/step - loss: 89.0100 - val_loss: 629.0407
Epoch 1476/10000
5/5 [==============================] - 0s 6ms/step - loss: 92.2792 - val_loss: 683.0461
Epoch 1477/10000
5/5 [==============================] - 0s 6ms/step - loss: 102.7997 - val_loss: 515.6087
Epoch 1478/10000
5/5 [==============================] - 0s 6ms/step - loss: 90.8998 - val_loss: 604.8318
Epoch 1479/10000
5/5 [==============================] - 0s 6ms/step - loss: 89.6251 - val_loss: 676.1104
Epoch 1480/10000
5/5 [==============================] - 0s 6ms/step - loss: 86.7273 - val_loss: 636.0974
Epoch 1481/10000
5/5 [==============================]

Epoch 1550/10000
5/5 [==============================] - 0s 6ms/step - loss: 82.5454 - val_loss: 723.6198
Epoch 1551/10000
5/5 [==============================] - 0s 6ms/step - loss: 82.8867 - val_loss: 706.6091
Epoch 1552/10000
5/5 [==============================] - 0s 7ms/step - loss: 85.3716 - val_loss: 416.7684
Epoch 1553/10000
5/5 [==============================] - 0s 6ms/step - loss: 94.2281 - val_loss: 558.3892
Epoch 1554/10000
5/5 [==============================] - 0s 6ms/step - loss: 83.0844 - val_loss: 862.8246
Epoch 1555/10000
5/5 [==============================] - 0s 6ms/step - loss: 84.1352 - val_loss: 670.3320
Epoch 1556/10000
5/5 [==============================] - 0s 6ms/step - loss: 94.2701 - val_loss: 677.6365
Epoch 1557/10000
5/5 [==============================] - 0s 6ms/step - loss: 95.3792 - val_loss: 742.5878
Epoch 1558/10000
5/5 [==============================] - 0s 6ms/step - loss: 87.2591 - val_loss: 686.8889
Epoch 1559/10000
5/5 [==============================] -

In [29]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [30]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,30.988646,30.32295,29.478382,29.357288,29.070728,28.560095,28.073397,27.984396,27.941139,27.884312,27.74082,27.864117,27.601357,27.564171,27.571186,27.599739,27.59071,27.430355,27.42297,27.421303,27.345831,27.334015,27.313591,27.30817,27.282593,27.19289,27.139351,27.033293,26.876877,1.686638
Target,53.235,59.29,54.875,57.52,49.573,40.698,47.195,43.527,45.488,44.96,36.721,54.463,51.462,58.854,53.94,51.514,42.635,40.371,48.99,44.726,44.491,43.901,55.019,50.942,63.992,64.127,60.822,65.182,51.282,43.554
Error,22.246355,28.967051,25.396618,28.162712,20.502274,12.137907,19.121603,15.542604,17.54686,17.075687,8.980181,26.598885,23.860645,31.289829,26.368813,23.914261,15.044289,12.940643,21.567032,17.304699,17.14517,16.566986,27.70541,23.633831,36.709408,36.934109,33.682648,38.148705,24.405125,41.867363


In [31]:
display(mae)
display(mape)

23.712257

0.45788085

In [32]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [33]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[345.26624]] - Target[587.5449999999998]| =  Error: [[242.27875]]; MAPE:[[0.41235778]]
1/1 [==============================] - 0s 19ms/step
Ano-0: |Prediction[[329.5034]] - Target[586.845]| =  Error: [[257.34158]]; MAPE:[[0.43851715]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[137.21164]] - Target[348.95899999999995]| =  Error: [[211.74738]]; MAPE:[[0.6067973]]


[array([[242.27875]], dtype=float32),
 array([[257.34158]], dtype=float32),
 array([[211.74738]], dtype=float32)]

237.12256

0.48589072